In [50]:
import ccxt
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone

binance = ccxt.binance()

symbol = 'BTC/USDT'
timeframe = '5m'
limit = 1000

since_dt = datetime.now(timezone.utc) - timedelta(days=4*3 + 3) # 4 weeks (+3 to avoid weekends if i fetch index, like the dxy, where they dont work on weekends, and i just remove where dxy is Nan)
since = int(since_dt.timestamp() * 1000) # second in milliseconds

all_candles = []

while True:
    candles = binance.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit) # binance api from ccxt
    if not candles:
        break
    all_candles += candles

    since = candles[-1][0] + 1 # update since to last candle timestamp + 1 ms
    time.sleep(0.5)  # avoid hitting rate limits with api

# Convert to DataFrame (table)
df = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

In [51]:
import yfinance as yf

dxy_daily = yf.download(
    'DX-Y.NYB',
    start=df['timestamp'].min().date() - pd.Timedelta(days=3),  # to avoid weekends
    end=df['timestamp'].max().date() + pd.Timedelta(days=3),
    interval='1d',
    progress=False,
    auto_adjust=False
)

# Reset index to move 'Date' to a column
dxy_daily = dxy_daily.reset_index(drop=False)

# Because its multilevel, make it one level (like df)
if isinstance(dxy_daily.columns, pd.MultiIndex):
    dxy_daily.columns = [col[0] for col in dxy_daily.columns]

dxy_daily.rename(columns={'Date': 'date'}, inplace=True)
dxy_daily.rename(columns={'Close': 'dxy_close'}, inplace=True)

dxy_daily["date"] = pd.to_datetime(dxy_daily["date"]).dt.date
df["date"] = pd.to_datetime(df["timestamp"]).dt.date

print(dxy_daily)

df = pd.merge(df, dxy_daily[["date", "dxy_close"]], on="date", how="left")

df['dxy_close'] = df['dxy_close'].ffill()
df = df.dropna(subset=['dxy_close']).reset_index(drop=True)
df.rename(columns={'dxy_close': 'dxy'}, inplace=True)

print(dxy_daily)

          date  Adj Close  dxy_close       High        Low       Open  Volume
0   2025-09-04  98.349998  98.349998  98.440002  98.080002  98.150002       0
1   2025-09-05  97.769997  97.769997  98.260002  97.430000  98.239998       0
2   2025-09-08  97.449997  97.449997  97.940002  97.419998  97.790001       0
3   2025-09-09  97.790001  97.790001  97.820000  97.250000  97.379997       0
4   2025-09-10  97.779999  97.779999  97.930000  97.599998  97.830002       0
5   2025-09-11  97.540001  97.540001  98.089996  97.470001  97.790001       0
6   2025-09-12  97.550003  97.550003  97.860001  97.489998  97.519997       0
7   2025-09-15  97.300003  97.300003  97.699997  97.269997  97.629997       0
8   2025-09-16  96.629997  96.629997  97.389999  96.559998  97.339996       0
9   2025-09-17  96.870003  96.870003  97.070000  96.220001  96.620003       0
10  2025-09-18  97.349998  97.349998  97.599998  96.839996  96.930000       0
11  2025-09-19  97.639999  97.639999  97.809998  97.269997  97.3

In [52]:

# df['ema_9'] = df['close'].ewm(span=9, adjust=False).mean()

df['ema_21'] = df['close'].ewm(span=21, adjust=False).mean()

df['sma_50'] = df['close'].rolling(window=50).mean()

df.dropna(subset=['ema_21', 'sma_50'], inplace=True)

# potential indicator to experiment with
# ema_short - sma_long for crossovers
# close / sma_50 - 1 for capturing trends

In [53]:
def buy_col_ranged(row):
    if pd.isna(row.next_high) or pd.isna(row.next_low):
        return np.nan
    if row.next_low >= row.high:
        return 2
    elif row.next_high > row.high:
        return 1
    elif row.next_high > row.low:
        return 0
    else:
        return -1
def sell_col_ranged(row):
    if pd.isna(row.next_high) or pd.isna(row.next_low):
        return np.nan
    if row.next_high <= row.low:
        return 2
    elif row.next_low < row.low:
        return 1
    elif row.next_low < row.high:
        return 0
    else:
        return -1

In [54]:
def buy_col(row):
    if pd.isna(row.next_high) or pd.isna(row.next_low):
        return np.nan
    return (row.next_high - row.high) / row.high

def sell_col(row):
    if pd.isna(row.next_high) or pd.isna(row.next_low):
        return np.nan
    return (row.low - row.next_low) / row.low

def define_activity(row):
    if row.signal_scaled < 0.2:
        return 'Strong Sell'
    elif row.signal_scaled < 0.4:
        return 'Sell'
    elif row.signal_scaled < 0.6:
        return 'Hold'
    elif row.signal_scaled < 0.8:
        return 'Buy'
    else:
        return 'Strong Buy'

In [55]:

df['next_high'] = df['high'].shift(-1)
df['next_low'] = df['low'].shift(-1)

df['buy_index'] = df.apply(buy_col, axis=1)
df['sell_index'] = df.apply(sell_col, axis=1)

df['signal'] = df['buy_index'] - df['sell_index']
scale = df['signal'].abs().median()
df['signal_scaled'] = 0.5 * (np.tanh( (df['buy_index'] - df['sell_index']) / scale ) + 1)

df['activity'] = df.apply(define_activity, axis=1)

df.dropna(subset=['buy_index','sell_index'], inplace=True)


In [56]:
print(df)

export_df = df[['dxy', 'ema_21', 'sma_50', 'activity']]
print(export_df)

%store export_df

               timestamp       open       high        low      close  \
49   2025-09-08 04:05:00  111042.48  111118.15  110953.06  111040.00   
50   2025-09-08 04:10:00  111039.99  111067.26  110950.00  111067.26   
51   2025-09-08 04:15:00  111067.26  111093.16  111045.53  111090.38   
52   2025-09-08 04:20:00  111090.39  111135.77  111065.66  111135.77   
53   2025-09-08 04:25:00  111135.77  111135.77  111118.00  111135.76   
...                  ...        ...        ...        ...        ...   
4273 2025-09-22 20:05:00  112077.58  112087.66  111975.28  112014.09   
4274 2025-09-22 20:10:00  112014.10  112137.31  112006.49  112127.20   
4275 2025-09-22 20:15:00  112127.21  112153.38  112044.04  112121.95   
4276 2025-09-22 20:20:00  112121.95  112155.50  112092.98  112155.49   
4277 2025-09-22 20:25:00  112155.48  112295.64  112155.48  112280.01   

        volume        date        dxy         ema_21       sma_50  next_high  \
49    97.64008  2025-09-08  97.449997  111094.219386  1